## Tối ưu hàm mục tiêu:
$$
\boxed{
\min_{G} E_{z \sim p_z} [\mathcal{L}_{\text{GAN}}(G(z)) + \lambda \cdot Q(G(z))]
}
$$
Trong đó:
* $G(z)$: ảnh được sinh từ latent vector $z$
* $Q(G(z))$: hàm đánh giá **chất lượng ảnh sinh**
* $\lambda$: hệ số điều chỉnh mức phạt của loss chất lượng

### Thay vì chỉ dùng $X$ là độ nhiễu và $Y$ là mức độ thật (mà không rõ đo thế nào), ta định nghĩa lại như sau:

**a. Feature distance $D_r$ – Độ khác biệt với ảnh thật:**

* Cho ảnh sinh $I_{gen}$, và ảnh thật tương ứng $I_{real}$
* Ta tính:
$$
D_r = \| \phi(I_{gen}) - \phi(I_{real}) \|_2
$$

Trong đó $\phi(\cdot)$ là feature extractor (VD: tầng giữa của VGG16 hoặc ResNet)

$\to$ **$D_r$ càng nhỏ thì ảnh càng giống thật**

**b. Noise-level estimator $N_g$ – Mức độ nhiễu nội tại:**

* Dựa trên thống kê gradient hoặc Laplacian:
$$
N_g = \text{Var}(\nabla I_{gen}) \quad (\text{hoặc}) \quad \text{Laplacian-based energy}
$$

$\to$ **Càng nhỏ thì ảnh càng mượt, ít nhiễu**

## Hàm đánh giá chất lượng tổng hợp
$$
Q(I_{gen}) = \alpha \cdot \text{soft}(N_g) + \beta \cdot \text{soft}(D_r)
$$

* $\text{soft}(\cdot)$ là hàm chuẩn hóa tuyến tính về khoảng [0,1]
* $\alpha, \beta$: trọng số học được hoặc chọn dựa vào yêu cầu (ví dụ: penalize nhiễu mạnh hơn)

$\to$ **Càng nhỏ, ảnh càng tốt**

## Điều kiện đánh giá hoặc regularizer cho GAN
**a. Ngưỡng hóa (Thresholding):**
$$
Q(I_{gen}) < \tau
$$
$\to$ giống như cách bạn đề xuất ban đầu, nhưng với các thành phần có định nghĩa rõ ràng hơn.

**b. Dùng làm Regularizer trong loss GAN:**
$$
\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{GAN}} + \lambda \cdot Q(I_{gen})
$$
$\to$ ép Generator sinh ảnh có **nhiễu thấp** và **giống thật** trong feature space.

## Mô hình tổng quát
$$
\boxed{
Q(I_{gen}) = \alpha \cdot \text{Norm}(\text{NoiseLevel}(I_{gen})) + \beta \cdot \text{Norm}(\| \phi(I_{gen}) - \phi(I_{real}) \|)
}
$$

* **Ngưỡng đánh giá:**
    $$
    Q(I_{gen}) < \tau
    $$

* **Hoặc dùng trong loss:**
    $$
    \min_{G} E_{z \sim p_z} [\mathcal{L}_{\text{GAN}}(G(z)) + \lambda \cdot Q(G(z))]
    $$

**Hàm chuẩn hóa:**
$$
\text{Norm}(x) = \frac{x - \min(x)}{\max(x) - \min(x)}
$$
$\to$ Đưa cả hai thành phần về cùng thang đo [0,1]


**Vai trò các hệ số**
$$
\alpha + \beta = 1 \quad (\text{nếu cần chuẩn hóa trọng số})
$$
* $\alpha$: trọng số cho độ nhiễu $\to$ kiểm soát độ sắc nét
* $\beta$: trọng số cho độ giống thật $\to$ kiểm soát tính chân thực

In [30]:
import torchvision.utils as vutils

def show_generated_images(generator, extractor, epoch, alpha=0.5, beta=0.5, device='cpu'):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(16, latent_dim).to(device)
        fake_images = generator(z)
        
        # Lấy mẫu ảnh thật để so sánh
        real_batch = next(iter(dataloader))[0][:16].to(device)
        q_score = compute_quality(fake_images, real_batch, extractor, alpha, beta)

        grid = vutils.make_grid(fake_images, nrow=4, normalize=True)
        plt.figure(figsize=(5,5))
        plt.axis("off")
        plt.title(f"Generated Images - Epoch {epoch+1}\nQ(I) Score = {q_score.item():.4f}")
        plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
        plt.show()
    generator.train()


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import os

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# === Hyperparameters ===
latent_dim = 100
batch_size = 64
alpha = 0.5
beta = 0.5
lambda_q = 0.1  # đã giảm
epochs = 500

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === MNIST Dataset ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# === Generator ===
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 784),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z).view(-1, 1, 28, 28)

# === Discriminator ===
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x)

# === Feature Extractor ===
class FeatureExtractor(nn.Module):
    def __init__(self, D):
        super().__init__()
        self.features = nn.Sequential(*list(D.net.children())[:-2])

    def forward(self, x):
        return self.features(x)


In [32]:
# === Hàm tính noise & khoảng cách đặc trưng ===
def compute_noise_level(img):
    grad_x = img[:, :, :, 1:] - img[:, :, :, :-1]
    grad_y = img[:, :, 1:, :] - img[:, :, :-1, :]
    return (grad_x.abs().mean() + grad_y.abs().mean())

def compute_feature_distance(gen_img, real_img, extractor):
    feat_gen = extractor(gen_img)
    feat_real = extractor(real_img)
    return F.mse_loss(feat_gen, feat_real)

# === Chuẩn hóa động Q(I) ===
Q_MIN, Q_MAX = 1e10, -1e10

def compute_quality(gen_img, real_img, extractor, alpha=0.5, beta=0.5):
    global Q_MIN, Q_MAX
    noise = compute_noise_level(gen_img)
    dist = compute_feature_distance(gen_img, real_img, extractor)
    raw_q = alpha * noise + beta * dist

    Q_MIN = min(Q_MIN, raw_q.item())
    Q_MAX = max(Q_MAX, raw_q.item())

    return (raw_q - Q_MIN) / (Q_MAX - Q_MIN + 1e-8)


In [33]:
def show_and_save_generated_images(generator, extractor, epoch, device='cpu'):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(16, latent_dim).to(device)
        fake_images = generator(z)

        real_batch = next(iter(dataloader))[0][:16].to(device)
        q_score = compute_quality(fake_images, real_batch, extractor, alpha, beta)

        grid = vutils.make_grid(fake_images, nrow=4, normalize=True)
        plt.figure(figsize=(5,5))
        plt.axis("off")
        plt.title(f"Epoch {epoch+1} | Q(I): {q_score.item():.4f}")
        plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
        plt.savefig(f"gen_images/epoch_{epoch+1:03}_Q{q_score.item():.2f}.png")
        plt.close()
    generator.train()


In [34]:
# === Khởi tạo mô hình ===
G = Generator().to(device)
D = Discriminator().to(device)
feature_extractor = FeatureExtractor(D).to(device)

optim_G = torch.optim.Adam(G.parameters(), lr=1e-3)
optim_D = torch.optim.Adam(D.parameters(), lr=5e-4)  # giảm LR của D
loss_fn = nn.BCEWithLogitsLoss()

# === Tạo thư mục lưu ảnh
os.makedirs("gen_images", exist_ok=True)

G_losses, D_losses, Q_scores = [], [], []

# === Huấn luyện ===
for epoch in range(epochs):
    for real_imgs, _ in dataloader:
        real_imgs = real_imgs.to(device)

        # === Train Discriminator ===
        z = torch.randn(real_imgs.size(0), latent_dim).to(device)
        fake_imgs = G(z).detach()
        D_real = D(real_imgs)
        D_fake = D(fake_imgs)

        loss_D = loss_fn(D_real, torch.ones_like(D_real)) + \
                 loss_fn(D_fake, torch.zeros_like(D_fake))
        optim_D.zero_grad()
        loss_D.backward()
        optim_D.step()

        # === Train Generator ===
        z = torch.randn(real_imgs.size(0), latent_dim).to(device)
        gen_imgs = G(z)
        D_out = D(gen_imgs)

        loss_GAN = loss_fn(D_out, torch.ones_like(D_out))
        q_loss = compute_quality(gen_imgs, real_imgs, feature_extractor, alpha, beta)
        loss_G = loss_GAN + lambda_q * q_loss

        optim_G.zero_grad()
        loss_G.backward()
        optim_G.step()

    # === Ghi lại lịch sử
    G_losses.append(loss_G.item())
    D_losses.append(loss_D.item())
    Q_scores.append(q_loss.item())

    print(f"Epoch {epoch+1}: G_loss={loss_G.item():.4f}, D_loss={loss_D.item():.4f}, Q={q_loss.item():.4f}")
    show_and_save_generated_images(G, feature_extractor, epoch, device=device)


Epoch 1: G_loss=8.9516, D_loss=0.0002, Q=0.9831
Epoch 2: G_loss=9.3295, D_loss=0.0002, Q=0.9980
Epoch 3: G_loss=9.3272, D_loss=0.0002, Q=0.9342
Epoch 4: G_loss=7.3632, D_loss=0.0058, Q=0.6924
Epoch 5: G_loss=8.2406, D_loss=0.0013, Q=0.4385
Epoch 6: G_loss=10.0672, D_loss=0.0002, Q=0.3037
Epoch 7: G_loss=7.6032, D_loss=0.0991, Q=0.2132
Epoch 8: G_loss=6.1340, D_loss=0.0129, Q=0.0870
Epoch 9: G_loss=4.8662, D_loss=0.0669, Q=0.0848
Epoch 10: G_loss=4.8304, D_loss=0.1111, Q=0.0663
Epoch 11: G_loss=5.0419, D_loss=0.0615, Q=0.0502
Epoch 12: G_loss=5.0251, D_loss=0.1263, Q=0.0572
Epoch 13: G_loss=5.5433, D_loss=0.2938, Q=0.0879
Epoch 14: G_loss=3.5467, D_loss=0.1613, Q=0.0789
Epoch 15: G_loss=6.1461, D_loss=0.3516, Q=0.1135
Epoch 16: G_loss=6.7190, D_loss=0.2415, Q=0.0902
Epoch 17: G_loss=5.3996, D_loss=0.7146, Q=0.0892
Epoch 18: G_loss=5.2538, D_loss=1.5608, Q=0.1053
Epoch 19: G_loss=4.3071, D_loss=1.2282, Q=0.1051
Epoch 20: G_loss=3.5170, D_loss=1.4844, Q=0.1160
Epoch 21: G_loss=3.0661, D_l

In [35]:
import imageio
import glob

images = []
for filename in sorted(glob.glob("gen_images/*.png")):
    images.append(imageio.imread(filename))
imageio.mimsave("gan_training.gif", images, fps=2)


/var/folders/5m/ngsgzxjs7yz5k08m054bfl700000gn/T/ipykernel_90940/523632294.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
